In [3]:
import sys
sys.path.append('../')

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
%reload_ext autoreload

In [64]:
from scripts.detect_voids import get_neightbours
from packages.yolo_predict import YOLO_Pred
import pandas as pd
import cv2

In [13]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [95]:
df_predictions = yolo.predictions('/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [98]:
df_predictions.loc[187]

Center_X    329.663025
Center_Y    538.918457
Width        24.987236
Height       23.199200
Name: 187, dtype: float64

In [86]:
dict_of_neightbours = get_neightbours(img_path='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [87]:
dict_of_neightbours[0]

{'572': [571, 564, 273, 277, 275, 599, 600],
 '1059': [1054, 825, 1055],
 '581': [576, 577, 580, 579, 317, 20],
 '585': [581, 583, 580, 579, 20, 612],
 '1203': [1201, 430, 166, 428, 1198],
 '576': [572, 574, 571, 277],
 '432': [431, 430, 428, 1198, 1145],
 '1067': [581, 576, 577, 583, 574, 580, 1297, 1294, 1060, 579, 1296],
 '577': [572, 576, 574, 571],
 '583': [581, 577, 580, 579, 317, 20],
 '574': [572, 571, 277, 275],
 '1172': [432, 431, 430, 1145],
 '580': [576, 577, 574, 579, 317],
 '1205': [1203, 1201, 166, 428],
 '1054': [1053, 234],
 '1297': [1294, 1060, 1296],
 '496': [494, 1235, 1237],
 '571': [564, 273, 561, 277, 270, 275, 562, 304, 599, 600],
 '494': [1235, 1237, 1233],
 '586': [581, 585, 583, 580, 20, 612],
 '1294': [572, 1059, 576, 574, 571, 1060, 825, 277, 275],
 '431': [430, 428, 1198, 1145],
 '1235': [1233],
 '1237': [1235, 1233],
 '834': [1067, 1297, 1296],
 '89': [399, 1150, 1152, 99],
 '1060': [1059, 825, 273, 277, 275, 1055],
 '1201': [166, 1198],
 '459': [1203, 12

In [88]:
len(df_predictions)

161

In [ ]:
for index in df_predictions.index:
    print(index)
    for list_ in dict_of_neightbours[0][str(index)]:
        print(list_)

In [89]:
len(dict_of_neightbours[0])

152

In [23]:
dict_of_neightbours[0]['1145']

KeyError: '1145'

In [63]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [82]:
def bb_intersection_over_union(df_predictions: pd.DataFrame, index_key:int, index_neightbour:int) -> float:

    a = index_key
    xA1 = df_predictions.loc[a][0] - df_predictions.loc[a][2]/2
    yA1 = df_predictions.loc[a][1] + df_predictions.loc[a][3]/2
    xA2 = df_predictions.loc[a][0] + df_predictions.loc[a][2]/2
    yA2 = df_predictions.loc[a][1] - df_predictions.loc[a][3]/2
    boxA = [xA1, yA1, xA2, yA2 ]

    b = index_neightbour # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0] - df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1] + df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0] + df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1] - df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# para left w = -w y h = 0
# para right w = +w y h = 0
# para top w = 0 y h = + h

In [90]:
df_predictions_test = df_predictions.copy()

In [94]:
df_predictions_test.loc[1260]

Center_X             194.691193
Center_Y             139.134720
Width                 22.782619
Height                90.207382
Height_over_image    184.238411
Name: 1260, dtype: float64

In [91]:
df_predictions_test['Height_over_image'] =  df_predictions_test.Center_Y + (df_predictions_test.Height/2)

In [92]:
df_predictions_test[df_predictions_test['Height_over_image'] == df_predictions_test['Height_over_image'].max()]

,Center_X,Center_Y,Width,Height,Height_over_image
1250,18.964607,563.583618,26.012243,46.716412,586.941824


In [60]:
df_predictions.loc[572][2]

23.51447868347168

In [65]:
image = cv2.imread(filename='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [72]:
image.shape[0:2]

(3264, 2448)

In [99]:
for index_a, index_b in dict_of_neightbours[0].items():
    print(f'This bb {index_a} has neightbours')
    h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
    w_index_a = df_predictions.loc[index_a][2]
    xA1 = df_predictions.loc[index_a][0] - df_predictions.loc[index_a][2]/2 - w_index_a
    yA1 = df_predictions.loc[index_a][1] + df_predictions.loc[index_a][3]/2
    xA2 = df_predictions.loc[index_a][0] + df_predictions.loc[index_a][2]/2 - w_index_a
    yA2 = df_predictions.loc[index_a][1] - df_predictions.loc[index_a][3]/2
    boxA = [xA1, yA1, xA2, yA2]
    if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
        for item in index_b:
            print(f'Neightbour bound box: {item}')
            xB1 = df_predictions.loc[index_b][0] - df_predictions.loc[index_b][2]/2
            yB1 = df_predictions.loc[index_b][1] + df_predictions.loc[index_b][3]/2
            xB2 = df_predictions.loc[index_b][0] + df_predictions.loc[index_b][2]/2
            yB2 = df_predictions.loc[index_b][1] - df_predictions.loc[index_b][3]/2
            boxB = [xB1, yB1,xB2, yB2 ]

            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])

            interArea = (xB - xA) * (yB - yA)

            boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
            boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

            iou = interArea / float(boxAArea + boxBArea - interArea)
            if iou <= 0.05:
                print(f'Entre {index_b} y {index_a} hay un espacio vacio')






This bb 572 has neightbours


KeyError: '572'

In [ ]:
a = index_a
xA1 = df_predictions.loc[a][0]-df_predictions.loc[a][2]/2
yA1 = df_predictions.loc[a][1]+df_predictions.loc[a][3]/2
xA2 = df_predictions.loc[a][0]+df_predictions.loc[a][2]/2
yA2 = df_predictions.loc[a][1]-df_predictions.loc[a][3]/2
boxA = [xA1, yA1, xA2, yA2 ]

b = index_b # indices vecinos detectados que estan en el diccionario
xB1 = df_predictions.loc[b][0]-df_predictions.loc[b][2]/2
yB1 = df_predictions.loc[b][1]+df_predictions.loc[b][3]/2
xB2 = df_predictions.loc[b][0]+df_predictions.loc[b][2]/2
yB2 = df_predictions.loc[b][1]-df_predictions.loc[b][3]/2
boxB = [xB1, yB1,xB2, yB2 ]
# determine the (x, y)-coordinates of the intersection rectangle
xA = max(boxA[0], boxB[0])
yA = max(boxA[1], boxB[1])
xB = min(boxA[2], boxB[2])
yB = min(boxA[3], boxB[3])


# compute the area of intersection rectangle
interArea = (xB - xA) * (yB - yA)

# compute the area of both the prediction and ground-truth
# rectangles
boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

# compute the intersection over union by taking the intersection
# area and dividing it by the sum of prediction + ground-truth
# areas - the interesection area
iou = interArea / float(boxAArea + boxBArea - interArea)

In [ ]:
iou